# My naive script

In [1]:
import os
from datetime import datetime
import numpy as np
import pandas as pd
import xarray as xr

In [2]:
xr.set_options(display_expand_attrs=False)

# Get CLM6 and CLM5 parameter ranges

In [3]:
full_clm5_csv = pd.read_csv("clm5_paramranges.csv")

clm5_csv = pd.read_csv("clm5_paramranges.csv")
clm6_csv = pd.read_csv("ctsm6lhc_paramranges_11262024.csv")

defparam_claire = xr.open_dataset('/glade/u/home/czarakas/coupled_PPE/data/paramfiles/OAAT0000.nc', decode_timedelta=False)

param_in_crosswalk = list(pd.read_csv("coupPPE-hist_crosswalk_simple.csv").param.unique())

clm5_csv = clm5_csv.rename(columns={"name":"param", "location":"loc"})
clm5_csv = clm5_csv[["param","loc","flag","min","max","pft_mins","pft_maxs"]]
clm5_subset = clm5_csv[clm5_csv["param"].isin(param_in_crosswalk)].reset_index(drop=True)

clm6_csv = clm6_csv.drop(columns="include").sort_values(by="param")
clm6_subset = clm6_csv[clm6_csv["param"].isin(param_in_crosswalk)].reset_index(drop=True)

for p in param_in_crosswalk:
    if p in list(clm6_csv["flag"]):
        tmp = clm6_csv[clm6_csv["flag"]==p]
        tmp = tmp[tmp["param"]!=p]
        clm6_subset = pd.concat([clm6_subset, tmp])

        tmp = clm5_csv[clm5_csv["param"].isin(tmp["param"])]
        tmp.loc[:, "flag"] = p
        clm5_subset = pd.concat([clm5_subset, tmp])
        clm5_subset.loc[len(clm5_subset)] = [p, p, np.nan, np.nan, np.nan, np.nan, np.nan]

clm5_subset = clm5_subset.sort_values(by="param").reset_index(drop=True)
clm6_subset = clm6_subset.sort_values(by="param").reset_index(drop=True)

param_in_clm5 = sorted(clm5_subset.param)
param_in_clm6 = sorted(clm6_subset.param)

In [4]:
# clm5_subset

In [5]:
# clm6_subset

In [6]:
print(len(param_in_crosswalk))
param_in_crosswalk

16


['default',
 'fff',
 'jmaxb0',
 'medlynslope',
 'medlynintercept',
 'maximum_leaf_wetted_fraction',
 'kmax',
 'leafcn',
 'd_max',
 'psi50',
 'cv',
 'lmrha',
 'FUN_fracfixers',
 'KCN',
 'ACCLIM_SF',
 'sand_pf']

In [7]:
print(len(param_in_clm5))
print(set(param_in_crosswalk) - set(param_in_clm5))
# param_in_clm5

24
{'default'}


In [8]:
print(len(param_in_clm6))
print(set(param_in_crosswalk) - set(param_in_clm6))
# param_in_clm6

23
{'sand_pf', 'default'}


# Create comprehensive crosswalk for coupPPE-hist
No need to run again, just load `coupPPE-hist_crosswalk.csv`

In [46]:
crosswalk = pd.read_csv("coupPPE-hist_crosswalk_simple.csv")
nkey = []
for key in crosswalk.key:
    nkey.append(str(key).zfill(3))
crosswalk["key"] = nkey

# Add columns to hold the perturbation values
crosswalk["pert"] = "NaN"
crosswalk["pft_pert"] = "NaN"

# Add the perturbation values
for index, row in crosswalk.iterrows():
    if row["origin_PPE"] == "CLM5":
        # min
        mask = (crosswalk.param == row.param) & (crosswalk.minmax == "min")
        value = clm5_subset.loc[clm5_subset.param == row.param, "min"]
        if not value.empty:
            crosswalk.loc[mask, "pert"] = value.values[0]
        value = clm5_subset.loc[clm5_subset.param == row.param, "pft_mins"]
        if not value.empty:
            crosswalk.loc[mask, "pft_pert"] = value.values[0]
        
        # max
        mask = (crosswalk.param == row.param) & (crosswalk.minmax == "max")
        value = clm5_subset.loc[clm5_subset.param == row.param, "max"]
        if not value.empty:
            crosswalk.loc[mask, "pert"] = value.values[0]
        value = clm5_subset.loc[clm5_subset.param == row.param, "pft_maxs"]
        if not value.empty:
            crosswalk.loc[mask, "pft_pert"] = value.values[0]
    
    elif row["origin_PPE"] == "CLM6":
        # min
        mask = (crosswalk.param == row.param) & (crosswalk.minmax == "min")
        value = clm6_subset.loc[clm6_subset.param == row.param, "min"]
        if not value.empty:
            crosswalk.loc[mask, "pert"] = value.values[0]
        value = clm6_subset.loc[clm6_subset.param == row.param, "pft_mins"]
        if not value.empty:
            crosswalk.loc[mask, "pft_pert"] = value.values[0]
        
        # max
        mask = (crosswalk.param == row.param) & (crosswalk.minmax == "max")
        value = clm6_subset.loc[clm6_subset.param == row.param, "max"]
        if not value.empty:
            crosswalk.loc[mask, "pert"] = value.values[0]
        value = clm6_subset.loc[clm6_subset.param == row.param, "pft_maxs"]
        if not value.empty:
            crosswalk.loc[mask, "pft_pert"] = value.values[0]

    else:
        print("Not in CLM5 or CLM6 subsets:", row.param)

# Identify individual parameters from groupings
grouped = []
for index, row in clm5_subset.iterrows():
    if row.flag == "ACCLIM_SF":
        grouped.append(row.to_frame().T)
    if row.flag == "KCN":
        grouped.append(row.to_frame().T)
grouped = pd.concat(grouped).sort_values(by="flag")

# Add grouped parameters to the crosswalk
for index, row in grouped.iterrows():
    description = full_clm5_csv.loc[full_clm5_csv.name==row.param].Description.item()
    category = full_clm5_csv.loc[full_clm5_csv.name==row.param].category.item()
    subcategory = full_clm5_csv.loc[full_clm5_csv.name==row.param].subcat.item()
    crosswalk.loc[len(crosswalk)] = [np.nan, row["param"], row["loc"], "min", row["flag"], description, category, subcategory, "CLM5", np.nan, row["min"], row["pft_mins"]]
    crosswalk.loc[len(crosswalk)] = [np.nan, row["param"], row["loc"], "max", row["flag"], description, category, subcategory, "CLM5", np.nan, row["max"], row["pft_maxs"]]

# crosswalk.to_csv("coupPPE-hist_crosswalk.csv")

Not in CLM5 or CLM6 subsets: sand_pf
Not in CLM5 or CLM6 subsets: sand_pf


In [9]:
# crosswalk

# Check how percent perturbations are implemented

In [10]:
# When the min and max are both listed as 50percent
p = "lmrha"
indir = "/glade/u/home/czarakas/coupled_PPE/data/paramfiles"

pmin = xr.open_dataset(f"{indir}/OAAT0355.nc", decode_timedelta=False)[p].values
pmax = xr.open_dataset(f"{indir}/OAAT0356.nc", decode_timedelta=False)[p].values
pdef = xr.open_dataset(f"{indir}/OAAT0000.nc", decode_timedelta=False)[p].values

print("min", pmin)
print("max", pmax)
print("def", pdef)

print("max/def =", pmax / pdef)
print("min/def =", pmin / pdef)

min 23195.0
max 69585.0
def 46390.0
max/def = 1.5
min/def = 0.5


In [11]:
# When the min and max are both listed as 20percent
p = "rhosnir"
indir = "/glade/u/home/czarakas/coupled_PPE/data/paramfiles"

pmin = xr.open_dataset(f"{indir}/OAAT0013.nc", decode_timedelta=False)[p].values
pmax = xr.open_dataset(f"{indir}/OAAT0014.nc", decode_timedelta=False)[p].values
pdef = xr.open_dataset(f"{indir}/OAAT0000.nc", decode_timedelta=False)[p].values

print("min", pmin[-1])
print("max", pmax[-1])
print("def", pdef[-1])

print("max/def =", pmax[-1] / pdef[-1])
print("min/def =", pmin[-1] / pdef[-1])

min 0.42400000000000004
max 0.636
def 0.53
max/def = 1.2
min/def = 0.8


In [12]:
# When the min is 50percent and max is 200percent
p = "tpu25ratio"
indir = "/glade/u/home/czarakas/coupled_PPE/data/paramfiles"

pmin = xr.open_dataset(f"{indir}/OAAT0137.nc", decode_timedelta=False)[p].values
pmax = xr.open_dataset(f"{indir}/OAAT0138.nc", decode_timedelta=False)[p].values
pdef = xr.open_dataset(f"{indir}/OAAT0000.nc", decode_timedelta=False)[p].values

print("min", pmin)
print("max", pmax)
print("def", pdef)

print("max/def =", pmax / pdef)
print("min/def =", pmin / pdef)

min 0.0835
max 0.501
def 0.167
max/def = 3.0
min/def = 0.5


# Generate parameter files (my implementation)

In [ ]:
print(str(datetime.now().strftime('%Y-%m-%dT%H:%M:%S')))

to_copy_defparam = False
to_save = True

# Where to save the parameter files
pdir = '/glade/u/home/bbuchovecky/projects/cpl_ppe_co2/hist/data/paramfiles'

# Where to save the namelist mods
ndir = '/glade/u/home/bbuchovecky/projects/cpl_ppe_co2/hist/data/namelist_mods'

# Open the crosswalk which includes all parameters and their min/max perturbations
crosswalk = pd.read_csv("coupPPE-hist_crosswalk.csv", index_col=0, dtype=str)

# Load the default parameter file
# Copied from /glade/u/home/czarakas/coupled_PPE/data/paramfiles
if to_copy_defparam:
    odefpfile = '/glade/u/home/bbuchovecky/projects/cpl_ppe_co2/hist/data/paramfiles/COUP0000.nc'
    defpfile = xr.open_dataset(odefpfile, decode_timedelta=False)

    now = str(datetime.now().strftime('%Y-%m-%dT%H:%M:%S'))
    defpfile.attrs = {
        'key': '000',
        'param': 'default',
        'minmax': 'default',
        'description': '',
        'category': '',
        'subcategory': '',
        'crosswalk': '/glade/u/home/bbuchovecky/projects/cpl_ppe_co2/hist/code/02_set-up-ensemble/coupPPE-hist_crosswalk.csv',
        'modified': now,
        'modified_by': 'bbuchovecky',
        'notes': 'coupled PPE with transient historical forcing and prescribed SST',
    } | defpfile.attrs
    defpfile.to_netcdf(f"{pdir}/param.coupPPE-hist.000.nc")

else:
    defpfile = xr.open_dataset(f"{pdir}/param.coupPPE-hist.000.nc", decode_timedelta=False)

# Iterate through the crosswalk and generate the perturbed parameter files
crosswalk_pert = crosswalk.loc[crosswalk["param"]!="default"]
newpfile_dict = dict()

print("|", "KEY", "|", "MINMAX".ljust(7), "|", "PARAM".ljust(28))
print("".ljust(46, "-"))
for index, row in crosswalk_pert.iterrows():
    key = row["key"]
    param = row["param"]
    minmax = row["minmax"]
    ploc = row["loc"]

    # Only choose core parameters, not including the component grouped parameters
    if (row["flag"] != "KCN") and (row["flag"] != "ACCLIM_SF"):
        print("|", key, "|", minmax.ljust(7), "|", param.ljust(28))

        # Create a deep copy of the default parameter set
        newpfile = defpfile.copy(deep=True)

        ### DO PARAMETER FILES

        # Perturb the parameter file if the parameter is not located in the namelist
        if ploc != "N":

            # Get the component grouped parameters if necessary
            plist = [param]
            if row["loc"].item() in ("KCN", "ACCLIM_SF"):
                grouped = row["loc"]
                plist = list(crosswalk.loc[crosswalk["flag"]==grouped]["param"])
                print("> ", end="")
                for p in plist:
                    print(p, end=", ")
                print()
            
            for p in plist:
                prow = crosswalk.loc[(crosswalk["param"]==p) & (crosswalk["minmax"]==minmax)]
                pert = prow["pert"]
                pft_pert = prow["pft_pert"]

                ## GET THE PERTURBATION VALUE(S) FROM THE CROSSWALK
            
                # Deal with pft-level parameters
                if pert.values == 'pft':
                    pertdescr = np.array(pft_pert.values[0].split(','))

                    # Deal with percent change perturbation
                    if isinstance(pertdescr[0], str) and ('percent' in pertdescr[0]):
                        print("  ", p, "is pft percent perturbation")
                        pertval = np.zeros(len(pertdescr))
                        for i, ppft in enumerate(pertdescr):
                            pertpct = float(ppft.removesuffix('percent')) / 100
                            if minmax == "min":
                                assert (1 - pertpct) > 0, f"min percent perturbation is negative, {1 - pertpct}"
                                pertval[i] = ((1 - pertpct) * defpfile[p][:, len(pertdescr)]).values
                            if minmax == "max":
                                assert (1 + pertpct) > 0, f"min percent perturbation is negative, {1 - pertpct}"
                                pertval[i] = ((1 + pertpct) * defpfile[p][:, len(pertdescr)]).values
                            else:
                                raise ValueError()

                    # Deal with explicit value perturbation
                    else:
                        print("  ", p, "is pft perturbation")
                        pertval = pertdescr.astype(float)
                    
                # Deal with single parameter values
                else:
                    pertdescr = pert.values[0]
            
                    # Deal with percent change perturbation
                    if isinstance(pertdescr, str) and ('percent' in pertdescr):
                        print("  ", p, "is single percent perturbation")
                        pertpct = float(pertdescr.removesuffix('percent')) / 100
                        if minmax == "min":
                            assert (1 - pertpct) > 0, f"min percent perturbation is negative, {1 - pertpct}"
                            pertval = ((1 - pertpct) * defpfile[p]).values
                        elif minmax == "max":
                            assert (1 + pertpct) > 0, f"max percent perturbation is negative, {1 - pertpct}"
                            pertval = ((1 + pertpct) * defpfile[p]).values
                        else:
                            print(minmax)
                            raise ValueError()

                    # Deal with explicit value perturbation
                    else:
                        print("  ", p, "is single value perturbation")
                        pertval = float(pertdescr)
                    
                ## ASSIGN THE PERTURBATION VALUE(S) TO THE NEW PARAMETER FILE
            
                # Deal with parameters that have greater than one dimensions (e.g., segment)
                if newpfile[p].size > 1:
                    # Case when perturbation value is a single float and constant across dimensions
                    # (only really applicable to medlynintercept)
                    if isinstance(pertval, float):
                        print("  ", tuple(newpfile[p].dims), "single float applied uniformly across dimensions")
                        newpfile[p] = (newpfile[p].dims, np.full(newpfile[p].shape, pertval))

                    # Simple case when shapes match
                    elif pertval.shape == newpfile[p].shape:
                        print("  ", tuple(newpfile[p].dims), "shapes match")
                        newpfile[p] = (newpfile[p].dims, pertval)

                    # Set via explicit indexing for case with segment dimension, apply the same perturbations across segment
                    elif 'segment' in newpfile[p].dims:
                        # Assumes the same values are applied across segments
                        print("  ", tuple(newpfile[p].dims), "perturb", len(pertval), "pfts across", len(newpfile[p].segment), "segments")
                        assert newpfile[p].shape == (4, 79), f"{p} in defparam has shape {newpfile[p].shape} with segment dimension"
                        newpfile[p][:, :len(pertval)] = np.stack([pertval]*4, 0)

                    # Set via explicit indexing for case with PFT dimension and shapes do not match
                    elif 'pft' in newpfile[p].dims:
                        print("  ", tuple(newpfile[p].dims), "shape mismatch, assign the same perturbation across pfts")
                        newpfile[p][:len(pertval)] = pertval

                    else:
                        raise ValueError()

                else:
                    try:
                        newpfile[p] = pertval
                    except:
                        print(f"    pertval:  {pertval}")
                        print(f"    defparam: {defpfile[p].values}")
                
            print("".ljust(46, "-"))

        now = str(datetime.now().strftime('%Y-%m-%dT%H:%M:%S'))
        newpfile.attrs = {
            'key': key,
            'param': param,
            'minmax': minmax,
            'description': row.description,
            'category': row.category,
            'subcategory': row.subcategory,
            'crosswalk': '/glade/u/home/bbuchovecky/projects/cpl_ppe_co2/hist/code/02_set-up-ensemble/coupPPE-hist_crosswalk.csv',
            'modified': now,
            'modified_by': 'bbuchovecky',
            'notes': 'coupled PPE with transient historical forcing and prescribed SST',
        } | newpfile.attrs
        newpfile_dict[str(key).zfill(3)+","+param+','+minmax] = newpfile

        ### DO NAMELISTS
        
        if to_save:
            nml = "! user_nl_clm namelist options written by 01_generate-paramfiles.ipynb"
            with open(f"{ndir}/user_nl_clm.coupPPE-hist.{key}", "w") as f:
                if ploc == "N":
                    f.write(f"{nml} {now}:\n{param}={row.pert}")
                else:
                    f.write(f"{nml} {now}:")

if len(newpfile_dict) != 2 * len(crosswalk_pert[crosswalk_pert.flag!="KCN"].loc[crosswalk_pert.flag!="ACCLIM_SF"].param.unique()):
    print(f"\nnumber of parameter files is different than the number of intended perturbation!")
    print(len(newpfile_dict))
    print(2 * len(crosswalk_pert[crosswalk_pert.flag!="KCN"].loc[crosswalk_pert.flag!="ACCLIM_SF"].param.unique()))

if to_save:
    print("\nsaving to file")
    for dkey, da in newpfile_dict.items():
        key = dkey.split(",")[0]
        da.to_netcdf(f"{pdir}/param.coupPPE-hist.{key}.nc")

| key | minmax  | param                       
----------------------------------------------
| 001 | min     | fff                         
   fff is single value perturbation
----------------------------------------------
| 002 | max     | fff                         
   fff is single value perturbation
----------------------------------------------
| 003 | min     | jmaxb0                      
   jmaxb0 is single value perturbation
----------------------------------------------
| 004 | max     | jmaxb0                      
   jmaxb0 is single value perturbation
----------------------------------------------
| 005 | min     | medlynslope                 
   medlynslope is pft perturbation
   ('pft',) shapes match
----------------------------------------------
| 006 | max     | medlynslope                 
   medlynslope is pft perturbation
   ('pft',) shapes match
----------------------------------------------
| 007 | min     | medlynintercept             
   medlynintercept is sin

# Check that parameter files are correct

In [ ]:
# load default and perturbed parameters
# - dimensions are the same
# - compare perturbed values to crosswalk

In [98]:
crosswalk.loc[crosswalk["param"]==param]["loc"].unique()

array(['P'], dtype=object)

In [ ]:
def check_perturbed_param(param):
    """Compares the perturbed parameter file to the default parameter file."""
    np.set_printoptions(precision=4)

    def all_equal(lst):
        return all(x == lst[0] for x in lst)

    pdir = '/glade/u/home/bbuchovecky/projects/cpl_ppe_co2/hist/data/paramfiles'
    crosswalk = pd.read_csv("coupPPE-hist_crosswalk.csv", index_col=0, dtype=str)

    if crosswalk.loc[crosswalk["param"]==param]["loc"].unique().item() == "N":
        print("".ljust(50, "-"))
        print(param, "not located in parameter file")
        print("".ljust(50, "-"))
        return

    paramxc_min = crosswalk.loc[(crosswalk["param"]==param) & (crosswalk["minmax"]=="min")]
    paramxc_max = crosswalk.loc[(crosswalk["param"]==param) & (crosswalk["minmax"]=="max")]

    defpfile = xr.open_dataset(f"{pdir}/coupPPE-hist.000.nc", decode_timedelta=False)
    minpfile = xr.open_dataset(f"{pdir}/coupPPE-hist.{paramxc_min.key.item()}.nc", decode_timedelta=False)
    maxpfile = xr.open_dataset(f"{pdir}/coupPPE-hist.{paramxc_max.key.item()}.nc", decode_timedelta=False)
    prtpfile = {"min": minpfile, "max": maxpfile}

    # Get the component grouped parameters if necessary
    print("".ljust(50, "-"))
    print(param)
    plist = [param]
    if paramxc_min["loc"].item() in ("KCN", "ACCLIM_SF") and paramxc_max["loc"].item() == paramxc_min["loc"].item():
        grouped = paramxc_min["loc"].item()
        plist = list(crosswalk.loc[crosswalk["flag"] == grouped]["param"].unique())

    for p in plist:
        if len(plist) > 1:
            print("".ljust(50, "-"))
            print(">", p)
        
        defp = defpfile[p]

        pdims = [defp.dims]
        pshape = [defp.shape]

        for m in ["min", "max"]:
            print("".ljust(50, "-"))

            pxc = crosswalk.loc[crosswalk["param"]==p].loc[crosswalk["minmax"]==m]
            pert = pxc["pert"]
            pft_pert = pxc["pft_pert"]

            prtp = prtpfile[m][p]
            pdims.append(prtp.dims)
            pshape.append(prtp.shape)

            print(f"{m} | dim: {prtp.dims} | shape: {prtp.shape}")
            
            if pert.values == "pft":
                print("xc pft pert =", np.array(pft_pert.item().split(','), dtype=float))
                print("pft pert    =", prtp.values)
            else:
                print("xc pert     =", pert.item())
                print("pert        =", prtp.values)
        
            print("def         =", defp.values)
            print("pert/def    =", prtp.values / defp.values)

        assert all_equal(pdims), "mismatch dimensions"
        assert all_equal(pshape), "mismatch shapes"

In [137]:
import warnings

pdir = '/glade/u/home/bbuchovecky/projects/cpl_ppe_co2/hist/data/paramfiles'
ndir = '/glade/u/home/bbuchovecky/projects/cpl_ppe_co2/hist/data/namelist_mods'
crosswalk = pd.read_csv("coupPPE-hist_crosswalk.csv", index_col=0, dtype=str)

defpfile = xr.open_dataset(f"{pdir}/param.coupPPE-hist.000.nc", decode_timedelta=False)

crosswalk_pert = crosswalk.loc[crosswalk["param"]!="default"]

for param in crosswalk_pert["param"].unique():
    row = crosswalk_pert.loc[(crosswalk_pert["param"]==param) & (crosswalk_pert["minmax"]=="min")]
    if (row["flag"].item() != "KCN") and (row["flag"].item() != "ACCLIM_SF"):
        with warnings.catch_warnings():
            warnings.simplefilter("ignore", category=RuntimeWarning)
            check_perturbed_param(row["param"].item())

--------------------------------------------------
fff
--------------------------------------------------
min | dim: () | shape: ()
xc pert     = 0.02
pert        = 0.02
def         = 0.5
pert/def    = 0.04
--------------------------------------------------
max | dim: () | shape: ()
xc pert     = 5
pert        = 5.0
def         = 0.5
pert/def    = 10.0
--------------------------------------------------
jmaxb0
--------------------------------------------------
min | dim: () | shape: ()
xc pert     = 0.01
pert        = 0.01
def         = 0.0311
pert/def    = 0.3215434083601286
--------------------------------------------------
max | dim: () | shape: ()
xc pert     = 0.05
pert        = 0.05
def         = 0.0311
pert/def    = 1.6077170418006432
--------------------------------------------------
medlynslope
--------------------------------------------------
min | dim: ('pft',) | shape: (79,)
xc pft pert = [9.   1.53 1.53 1.53 1.7  1.7  2.05 2.05 2.05 0.91 0.91 0.91 3.78 3.78
 0.65 3.89 3.89

# Inspect Claire's namelist mods

In [2]:
import os
# directory = "/glade/u/home/czarakas/coupled_PPE/data/namelist_mods"
directory = "../../../data/namelist_mods"
for fname in sorted(os.listdir(directory)):
    fpath = os.path.join(directory, fname)
    if os.path.isfile(fpath) and fname.endswith('.txt'):
        with open(fpath, 'r') as f:
            content = f.read()
        print(f"{fname[:-4]}", end=" ")
        content = content.split("\n")
        if len(content) > 2:
            print(content[-2].split("=")[0], end="")
        print()

coupPPE-hist.000 
coupPPE-hist.001 
coupPPE-hist.002 
coupPPE-hist.003 
coupPPE-hist.004 
coupPPE-hist.005 
coupPPE-hist.006 
coupPPE-hist.007 
coupPPE-hist.008 
coupPPE-hist.009 maximum_leaf_wetted_fraction
coupPPE-hist.010 maximum_leaf_wetted_fraction
coupPPE-hist.011 
coupPPE-hist.012 
coupPPE-hist.013 
coupPPE-hist.014 
coupPPE-hist.015 
coupPPE-hist.016 
coupPPE-hist.017 
coupPPE-hist.018 
coupPPE-hist.019 
coupPPE-hist.020 
coupPPE-hist.021 
coupPPE-hist.022 
coupPPE-hist.023 
coupPPE-hist.024 
coupPPE-hist.025 
coupPPE-hist.026 
coupPPE-hist.027 
coupPPE-hist.028 
